<a href="https://colab.research.google.com/github/fowardelcac/Markowitz/blob/main/Notebooks/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import yfinance as yf
import scipy.optimize as sci_opt

In [5]:
def tickers(n):
  lista_tickers = []
  for i in range(n):
    ticker = str(input("Ingrese el ticker: "))
    lista_tickers.append(ticker.upper())
  return lista_tickers  

def descarga(lista, fecha = str):
  n = len(lista)
  assets_df = []
  for i in lista:
    df = yf.download(i, start = fecha)['Adj Close']
    assets_df.append(df)
  return assets_df

def calculos(weights: list):
  ret = np.sum(ret_log.mean() * weights) * 252
  vol = np.sqrt(np.dot(weights.T, np.dot(ret_log.cov() * 252, weights)))
  sr = ret / vol
  return np.array([ret, vol, sr])

def neg_s(weights: list) -> np.array:
  return calculos(weights)[2] - 1

def get_vol(weights: list)-> np.array:
  return calculos(weights)[1]

    
def montecarlo(n_iter, n_stocks):
  portfolio_returns, portfolio_volatilities, portfolio_sharpe  = [list() for _ in range(3)]
  all_weights = np.zeros((n_iter, n_stocks))

  for i in range(n_iter):
  
    weights = np.random.random(n_stocks)
    weights = weights / np.sum(weights)

    all_weights[i, :] = weights
    rdo = calculos(weights)
    portfolio_returns.append(rdo[0])
    portfolio_volatilities.append(rdo[1])
    portfolio_sharpe.append(rdo[2])

  return pd.DataFrame({
      'Return': portfolio_returns,
      'Volatility': portfolio_volatilities,
      'Sharpe': portfolio_sharpe,
      'Weights': all_weights.tolist()
      })

In [53]:
assets = tickers(3)
assets_df = descarga(assets, '2015-01-01')

df = pd.concat(assets_df, axis= 1)
df.columns = assets

df.dropna(inplace=True)

Ingrese el ticker: aapl
Ingrese el ticker: msft
Ingrese el ticker: tsla
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [9]:
ret_log = np.log(df / df.shift(1))

In [13]:
df_portafolio = montecarlo(1000, len(assets))
df_portafolio.head(2)

,Return,Volatility,Sharpe,Weights
0,0.265473,0.295511,0.898355,"[0.39067693956011335, 0.3558776010699778, 0.25..."
1,0.299881,0.403433,0.743322,"[0.006663502834718384, 0.38907787501247093, 0...."


In [15]:
max_sh = df_portafolio.iloc[df_portafolio.Sharpe.idxmax()]
min_vol = df_portafolio.iloc[df_portafolio.Volatility.idxmin()]
max_ret = df_portafolio.iloc[df_portafolio.Return.idxmax()]

In [28]:
fig = px.scatter(df_portafolio, x='Volatility', y='Return', color='Sharpe',
                 labels={'Volatility': 'Expected Volatility', 'Return': 'Expected Return'},
                 title='Portfolio Optimization',
                 hover_data={'Volatility': True, 'Return': True, 'Sharpe': True})

fig.add_scatter(x=[min_vol[1]], y=[min_vol[0]], mode='markers', marker=dict(color='red', symbol='star'), name='Minimum Expected Volatility')
fig.add_scatter(x=[max_ret[1]], y=[max_ret[0]], mode='markers', marker=dict(color='red', symbol='star'), name='Maximum Expected Return')
fig.add_scatter(x=[max_sh[1]], y=[max_sh[0]], mode='markers', marker=dict(color='red', symbol='square'), name='Maximum Sharpe Ratio')

fig.update_layout(
    coloraxis=dict(colorscale='plasma', colorbar=dict(title='Sharpe Ratio')),
    legend=dict(
        title=None,
        orientation='h',
        yanchor='bottom',
        y=-0.2,
        xanchor='right',
        x=1
    ),
    margin=dict(l=0, r=0, t=30, b=0),
    showlegend=True,
    hovermode='closest',
    plot_bgcolor='white',
    paper_bgcolor='white',
    xaxis=dict(showgrid=True, title_font=dict(size=12)),
    yaxis=dict(showgrid=True, title_font=dict(size=12)),
)

fig.show()


In [73]:
bounds = tuple((0, 1) for symbol in range(len(assets)))
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
init_guess = len(assets) * [1 / len(assets)]

optimized_sharpe = sci_opt.minimize(
    neg_s, 
    init_guess, 
    method='SLSQP',
    bounds=bounds, 
    constraints=constraints
)
      
optimized_metrics = calculos(weights=optimized_sharpe.x)
print('OPTIMIZED METRICS:')
print(optimized_metrics)
print('-'*80)
print('OPTIMIZED WEIGHTS:')
print(optimized_sharpe.x)
print('-'*80)

OPTIMIZED METRICS:
[0.334303   0.56873671 0.58779923]
--------------------------------------------------------------------------------
OPTIMIZED WEIGHTS:
[2.77555756e-17 2.77555756e-16 1.00000000e+00]
--------------------------------------------------------------------------------


In [78]:
data = df / df.iloc[0]
dff = pd.DataFrame()
indice = -1
for i in data:
    indice += 1
    dff[i] = (data[i] * optimized_sharpe.x[indice]) * 100000

dff['Value'] = dff.sum(axis=1)

In [80]:
dff

,AAPL,MSFT,TSLA,Value
Date,,,,
2015-01-02,0.00,0.00,100000.00,100000.00
2015-01-05,0.00,0.00,95795.90,95795.90
2015-01-06,0.00,0.00,96338.51,96338.51
2015-01-07,0.00,0.00,96188.03,96188.03
2015-01-08,0.00,0.00,96037.57,96037.57
...,...,...,...,...
2023-06-05,0.00,0.00,1488372.55,1488372.55
2023-06-06,0.00,0.00,1513679.17,1513679.17
2023-06-07,0.00,0.00,1535976.44,1535976.44


In [81]:
bounds = tuple((0, 1) for symbol in range(len(assets)))
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
init_guess = len(assets) * [1 / len(assets)]

optimized_sharpe = sci_opt.minimize(
    get_vol, 
    init_guess, 
    method='SLSQP',
    bounds=bounds, 
    constraints=constraints
)
      
optimized_metrics = calculos(weights=optimized_sharpe.x)
print('OPTIMIZED METRICS:')
print(optimized_metrics)
print('-'*80)
print('OPTIMIZED WEIGHTS:')
print(optimized_sharpe.x)
print('-'*80)

OPTIMIZED METRICS:
[0.24346023 0.26605274 0.9150826 ]
--------------------------------------------------------------------------------
OPTIMIZED WEIGHTS:
[0.41477629 0.58250258 0.00272113]
--------------------------------------------------------------------------------


In [83]:
data = df / df.iloc[0]
dff = pd.DataFrame()
indice = -1
for i in data:
    indice += 1
    dff[i] = (data[i] * optimized_sharpe.x[indice]) * 100000

dff['Value'] = dff.sum(axis=1)
dff

,AAPL,MSFT,TSLA,Value
Date,,,,
2015-01-02,41477.63,58250.26,272.11,100000.00
2015-01-05,40309.13,57714.59,260.67,98284.40
2015-01-06,40312.94,56867.50,262.15,97442.59
2015-01-07,40878.20,57590.03,261.74,98729.97
2015-01-08,42448.84,59284.24,261.33,101994.41
...,...,...,...,...
2023-06-05,303628.90,481739.89,4050.05,789418.84
2023-06-06,303003.32,478499.02,4118.91,785621.26
2023-06-07,300653.15,463728.78,4179.59,768561.52
